# [GoogleColaboratoryで気象庁の過去気象データをスクレイピングしてみた。](https://qiita.com/Cyber_Hacnosuke/items/122cec35d299c4d01f10)

## [■気象データの観測地点一覧　（2017年3月時点）](http://k-ichikawa.blog.enjoy.jp/etc/HP/htm/jmaP0.html)

In [ ]:
prec_no_dict = {"11": "宗谷地方", 
                "12": "上川地方", 
                "13": "留萌地方", 
                "14": "石狩地方", 
                "15": "空知地方", 
                "16": "後志地方", 
                "17": "網走・北見・紋別地方", 
                "18": "根室地方", 
                "19": "釧路地方", 
                "20": "十勝地方", 
                "21": "胆振地方", 
                "22": "日高地方", 
                "23": "渡島地方", 
                "24": "檜山地方", 
                "31": "青森県", 
                "32": "秋田県", 
                "33": "岩手県", 
                "34": "宮城県", 
                "35": "山形県", 
                "36": "福島県", 
                "40": "茨城県", 
                "41": "栃木県", 
                "42": "群馬県", 
                "43": "埼玉県", 
                "44": "東京都", 
                "45": "千葉県", 
                "46": "神奈川県", 
                "48": "長野県", 
                "49": "山梨県", 
                "50": "静岡県", 
                "51": "愛知県", 
                "52": "岐阜県", 
                "53": "三重県", 
                "54": "新潟県", 
                "55": "富山県", 
                "56": "石川県", 
                "57": "福井県", 
                "60": "滋賀県", 
                "61": "京都府", 
                "62": "大阪府", 
                "63": "兵庫県", 
                "64": "奈良県", 
                "65": "和歌山県", 
                "66": "岡山県", 
                "67": "広島県", 
                "68": "島根県", 
                "69": "鳥取県", 
                "71": "徳島県", 
                "72": "香川県", 
                "73": "愛媛県", 
                "74": "高知県", 
                "81": "山口県", 
                "82": "福岡県", 
                "83": "大分県", 
                "84": "長崎県", 
                "85": "佐賀県", 
                "86": "熊本県", 
                "87": "宮崎県", 
                "88": "鹿児島県", 
                "91": "沖縄県", 
                "99": "南極"}

In [ ]:
prec_no_dict["99"]

'南極'

In [ ]:
place_codeA = [14, 21, 23, 31, 32, 33, 35, 34, 36, 54, 56, 55, 48, 41, 57, 42, 43, 40, 52, 51, 49, 44, 45, 53, 50, 46, 68, 69, 61, 62, 60, 67, 66, 63, 65, 64, 73, 72, 74, 71, 81, 82, 85, 83, 84, 86, 88, 87, 91]
place_codeB = [47412, 47423, 47430, 47575, 47582, 47584, 47588, 47590, 47595, 47604, 47605, 47607, 47610, 47615, 47616, 47624, 47626, 47629, 47632, 47636, 47638, 47662, 47648, 47651, 47656, 47670, 47741, 47746, 47759, 47772, 47761, 47765, 47768, 47770, 47777, 47780, 47887, 47891, 47893, 47895, 47762, 47807, 47813, 47815, 47817, 47819, 47827, 47830, 47936]
place_name = ["札幌", "室蘭", "函館", "青森", "秋田", "盛岡", "山形", "仙台", "福島", "新潟", "金沢", "富山", "長野", "宇都宮", "福井", "前橋", "熊谷", "水戸", "岐阜", "名古屋", "甲府", "東京", "銚子", "津", "静岡", "横浜", "松江", "鳥取", "京都", "大阪", "彦根", "広島", "岡山", "神戸", "和歌山", "奈良", "松山", "高松", "高知", "徳島", "下関", "福岡", "佐賀", "大分", "長崎", "熊本", "鹿児島", "宮崎", "那覇"]   


In [ ]:
#place_codeA = [14]
#place_codeB = [47412]
#place_name = ["札幌"]   


In [ ]:
import requests
from bs4 import BeautifulSoup #ダウンロードしてなかったらpipでできるからやってね。
import csv

# URLで年と月ごとの設定ができるので%sで指定した英数字を埋め込めるようにします。
base_url = "http://www.data.jma.go.jp/obd/stats/etrn/view/daily_s1.php?prec_no=%s&block_no=%s&year=%s&month=%s&day=1&view=p1"

#取ったデータをfloat型に変えるやつ。(データが取れなかったとき気象庁は"/"を埋め込んでいるから0に変える)
def str2float(str):
  try:
    return float(str)
  except:
    return 0.0


if __name__ == "__main__":
  #都市を網羅します
  #最終的にデータを集めるリスト (下に書いてある初期値は一行目。つまり、ヘッダー。)
  All_list = [['都府県・地方名', '都市' , '年月日', '陸の平均気圧(hPa)', '海の平均気圧(hPa)', '降水量(mm)', '平均気温(℃)', '平均湿度(%)', '平均風速(m/s)', '日照時間(h)', '降雪(cm)', '天気概況(昼)(06:00-18:00)', '天気概況(夜)(18:00-翌日06:00)']]
  for place in place_name:
    #最終的にデータを集めるリスト (下に書いてある初期値は一行目。つまり、ヘッダー。)
    #All_list = [['都市' , '年月日', '陸の平均気圧(hPa)', '海の平均気圧(hPa)', '降水量(mm)', '平均気温(℃)', '平均湿度(%)', '平均風速(m/s)', '日照時間(h)']]
    print(place)
    index = place_name.index(place)
    pref_name = prec_no_dict[str(place_codeA[index])]
    print(pref_name)

    ### 過去分
    # for文で2007年~2017年までの11回。
    #for year in range(2007,2018):
    for year in range(2019,2021):
      print(year)
      # その年の1月~12月の12回を網羅する。
      for month in range(1,13):
        #2つの都市コードと年と月を当てはめる。
        r = requests.get(base_url%(place_codeA[index], place_codeB[index], year, month))
        r.encoding = r.apparent_encoding

        # まずはサイトごとスクレイピング
        soup = BeautifulSoup(r.text)
        # findAllで条件に一致するものをすべて抜き出します。
        # 今回の条件はtrタグでclassがmtxになってるものです。
        rows = soup.findAll('tr',class_='mtx')

        # 表の最初の1~4行目はカラム情報なのでスライスする。(indexだから初めは0だよ)
        # 【追記】2020/3/11 申し訳ございません。間違えてました。
        rows = rows[4:]

        # 1日〜最終日までの１行を網羅し、取得します。
        for row in rows:
          # 今度はtrのなかのtdをすべて抜き出します
          data = row.findAll('td')

          #１行の中には様々なデータがあるので全部取り出す。
          # ★ポイント
          rowData = [] #初期化
          rowData.append(pref_name)
          rowData.append(place)
          rowData.append(str(year) + "/" + str(month) + "/" + str(data[0].string))
          rowData.append(str2float(data[1].string))
          rowData.append(str2float(data[2].string))
          rowData.append(str2float(data[3].string))
          rowData.append(str2float(data[6].string))
          rowData.append(str2float(data[9].string))
          rowData.append(str2float(data[11].string))
          rowData.append(str2float(data[16].string))
          rowData.append(str2float(data[17].string))
          rowData.append(data[19].string)
          rowData.append(data[20].string)

          #次の行にデータを追加
          All_list.append(rowData)

    ### 最新分（該当月まで）
    for year in range(2021,2022):
      print(year)
      # その年の1月~12月の12回を網羅する。
      #for month in range(1,13):
      for month in range(1,9):
        #2つの都市コードと年と月を当てはめる。
        r = requests.get(base_url%(place_codeA[index], place_codeB[index], year, month))
        r.encoding = r.apparent_encoding

        # まずはサイトごとスクレイピング
        soup = BeautifulSoup(r.text)
        # findAllで条件に一致するものをすべて抜き出します。
        # 今回の条件はtrタグでclassがmtxになってるものです。
        rows = soup.findAll('tr',class_='mtx')

        # 表の最初の1~4行目はカラム情報なのでスライスする。(indexだから初めは0だよ)
        # 【追記】2020/3/11 申し訳ございません。間違えてました。
        rows = rows[4:]

        # 1日〜最終日までの１行を網羅し、取得します。
        for row in rows:
          # 今度はtrのなかのtdをすべて抜き出します
          data = row.findAll('td')

          #１行の中には様々なデータがあるので全部取り出す。
          # ★ポイント
          rowData = [] #初期化
          rowData.append(pref_name)
          rowData.append(place)
          rowData.append(str(year) + "/" + str(month) + "/" + str(data[0].string))
          rowData.append(str2float(data[1].string))
          rowData.append(str2float(data[2].string))
          rowData.append(str2float(data[3].string))
          rowData.append(str2float(data[6].string))
          rowData.append(str2float(data[9].string))
          rowData.append(str2float(data[11].string))
          rowData.append(str2float(data[16].string))
          rowData.append(str2float(data[17].string))
          rowData.append(data[19].string)
          rowData.append(data[20].string)

          #次の行にデータを追加
          All_list.append(rowData)

    #都市ごとにデータをファイルを新しく生成して書き出す。(csvファイル形式。名前は都市名)
    #with open(place + '.csv', 'w') as file:
    #  writer = csv.writer(file, lineterminator='\n')
    #  writer.writerows(All_list)

  #都市ごとにデータをファイルを新しく生成して書き出す。(csvファイル形式。名前は都市名)
  with open('result.csv', 'w') as file:
    writer = csv.writer(file, lineterminator='\n')
    writer.writerows(All_list)

札幌
石狩地方
2019
2020
2021
室蘭
胆振地方
2019
2020
2021
函館
渡島地方
2019
2020
2021
青森
青森県
2019
2020
2021
秋田
秋田県
2019
2020
2021
盛岡
岩手県
2019
2020
2021
山形
山形県
2019
2020
2021
仙台
宮城県
2019
2020
2021
福島
福島県
2019
2020
2021
新潟
新潟県
2019
2020
2021
金沢
石川県
2019
2020
2021
富山
富山県
2019
2020
2021
長野
長野県
2019
2020
2021
宇都宮
栃木県
2019
2020
2021
福井
福井県
2019
2020
2021
前橋
群馬県
2019
2020
2021
熊谷
埼玉県
2019
2020
2021
水戸
茨城県
2019
2020
2021
岐阜
岐阜県
2019
2020
2021
名古屋
愛知県
2019
2020
2021
甲府
山梨県
2019
2020
2021
東京
東京都
2019
2020
2021
銚子
千葉県
2019
2020
2021
津
三重県
2019
2020
2021
静岡
静岡県
2019
2020
2021
横浜
神奈川県
2019
2020
2021
松江
島根県
2019
2020
2021
鳥取
鳥取県
2019
2020
2021
京都
京都府
2019
2020
2021
大阪
大阪府
2019
2020
2021
彦根
滋賀県
2019
2020
2021
広島
広島県
2019
2020
2021
岡山
岡山県
2019
2020
2021
神戸
兵庫県
2019
2020
2021
和歌山
和歌山県
2019
2020
2021
奈良
奈良県
2019
2020
2021
松山
愛媛県
2019
2020
2021
高松
香川県
2019
2020
2021
高知
高知県
2019
2020
2021
徳島
徳島県
2019
2020
2021
下関
山口県
2019
2020
2021
福岡
福岡県
2019
2020
2021
佐賀
佐賀県
2019
2020
2021
大分
大分県
2019
2020
2021
長崎
長崎県
2019
2020
2021
熊本


In [ ]:
#from google.colab import files

#for place in place_name:
#  files.download(place + '.csv')


In [ ]:
from google.colab import files

files.download('result.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>